# Model 3 classical loudspeakers

Usefull for debugging 

References: 
- Mäkivirta, Aki & Liski, Juho & Välimäki, Vesa. (2018). Modeling and Delay-Equalizing Loudspeaker Responses. Journal of the Audio Engineering Society. 66. 922-934. 10.17743/jaes.2018.0053. 
- Linkwitz, Siegfried H. (February 1976). "AES E-Library: Browse Entire Database". JAES Volume 24 Issue 1. The Audio Engineering Society: 2–8. 

In [1]:
import math
import numpy as np
import pandas as pd
import altair as alt

In [2]:
freq = np.logspace(1+math.log10(2), 4+math.log10(2), 200)

In [3]:
import sys,os,os.path
sys.path.append(os.path.expanduser('../src'))

from spinorama.filter_iir import Biquad
from spinorama.filter_peq import peq_build

In [4]:
ls = Biquad(1, 20, 48000, 10, 10)
hs = Biquad(0, 20000, 48000, 10, 5)
filter_one_way = [(-1, ls), (-1, hs)]

In [5]:
one_way = peq_build(freq, filter_one_way)

In [6]:
alt.Chart(pd.DataFrame({'freq': freq, 'dB': one_way})).mark_line().encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('dB:Q', scale=alt.Scale(domain=[-40, 10]))
)

alt.Chart(...)

In [7]:
ls1 = Biquad(1,   80, 48000, 0.5, 10)
hs1 = Biquad(0, 5000, 48000, 0.5, 10)
ls2 = Biquad(1, 1000, 48000, 0.5, 10)
filter_woofer = [(1, ls1), (1, hs1)]
filter_tweeter = [(1, ls2)]

In [8]:
woofer_two_way = peq_build(freq, filter_woofer)
tweeter_two_way = peq_build(freq, filter_tweeter)

In [9]:
alt.Chart(
    pd.DataFrame({'freq': freq, 'woofer': woofer_two_way, 'tweeter': tweeter_two_way})
).transform_fold(['woofer', 'tweeter']
).transform_filter('datum.value>-40'
).mark_line(
).encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('value:Q', title='dB', scale=alt.Scale(domain=[-40, 10])),
    color='key:N',
)

alt.Chart(...)

In [10]:
from spinorama.filter_peq import peq_butterworth_lowpass, peq_butterworth_highpass, peq_linkwitzriley_lowpass, peq_linkwitzriley_highpass

In [11]:
bw2_lp = peq_build(freq, peq_butterworth_lowpass(2, 2000, 48000))
bw2_hp = peq_build(freq, peq_butterworth_highpass(2, 2000, 48000))
lr4_lp = peq_build(freq, peq_linkwitzriley_lowpass(4, 2000, 48000))
lr4_hp = peq_build(freq, peq_linkwitzriley_highpass(4, 2000, 48000))

In [12]:
alt.Chart(
    pd.DataFrame({'freq': freq, 'LR4 lp': lr4_lp, 'LR4 hp': lr4_hp, 'BW2 lp': bw2_lp+10, 'BW2 hp': bw2_hp+10})
).transform_fold(['LR4 lp', 'LR4 hp', 'BW2 lp', 'BW2 hp']
).transform_filter('datum.value>-40'
).mark_line().encode(
    x=alt.X('freq:Q', scale=alt.Scale(type="log", domain=[20, 20000], nice=False)), 
    y=alt.Y('value:Q', scale=alt.Scale(domain=[-40, 10], nice=False)),
    color='key:N',
)

alt.Chart(...)

In [13]:
import scipy.signal as sig

In [14]:
sig.besselap(4)

(array([], dtype=float64),
 array([-0.65721117+0.83016144j, -0.9047588 +0.27091873j,
        -0.9047588 -0.27091873j, -0.65721117-0.83016144j]),
 1.0)

In [60]:
z, p, k = sig.buttap(8)
np.real(p)

array([-0.19509032, -0.55557023, -0.83146961, -0.98078528, -0.98078528,
       -0.83146961, -0.55557023, -0.19509032])

In [16]:
1/math.sqrt(2)

0.7071067811865475

In [17]:
sos_lp = sig.butter(4, 0.125, btype='lp', output='sos')
sos_hp = sig.butter(4, 0.125, btype='hp', output='sos')

In [18]:
n = 201
t = np.linspace(0, 1, n)
np.random.seed(20210413)
x = 1 + (t < 0.5) - 0.25*t**2 + 0.05*np.random.randn(n)

In [68]:
y = sig.sosfiltfilt(sos_lp, x)
w_lp, h_lp = sig.sosfreqz(sos_lp, worN=n, fs=48000)
dB_lp = 20*np.log10(np.maximum(np.abs(h_lp), 1e-15))
w_hp, h_hp = sig.sosfreqz(sos_hp, worN=n, fs=48000)
dB_hp = 20*np.log10(np.maximum(np.abs(h_hp), 1e-15))
dB_sum = 20*np.log10(np.maximum(np.abs(h_lp+h_hp), 1e-15))

In [69]:
w_db = alt.Chart(
    pd.DataFrame({
        'freq': w_lp/math.pi, 
        'dB_lp': dB_lp, 'dB_hp': dB_hp, 'dB_sum': dB_sum
    }).melt('freq', var_name='buttw', value_name='dB')
).transform_filter('datum.dB>-9'
).mark_line(
).encode(
    x=alt.X('freq:Q', scale=alt.Scale(type='log', base=10, domain=[1, 20000], nice=False)),
    y=alt.Y('dB:Q', scale=alt.Scale(domain=[-9, 6], zero=False, nice=False)),
    color=alt.Color('buttw')
)

In [70]:
w_phase = alt.Chart(
    pd.DataFrame({'freq': w_lp/math.pi, 
                  'phase_lp': np.angle(h_lp)*180/math.pi,
                  'phase_hp': np.angle(h_hp)*180/math.pi,
                  'phase_sum': np.angle(h_lp+h_hp)*180/math.pi
                 }).melt('freq', var_name='buttw', value_name='phase')
).mark_line(
).encode(
    x=alt.X('freq:Q', scale=alt.Scale(type='log', base=10, domain=[1, 20000], nice=False)),
    y=alt.Y('phase:Q', axis=alt.Axis(title='Phase (deg)'), scale=alt.Scale(domain=[-180, 180], nice=False)),
    color=alt.Color('buttw')
)

In [71]:
x_y = alt.Chart(
    pd.DataFrame({'t':t, 'x': x, 'y': y}
    ).melt('t', var_name='signal', value_name='dB')
).mark_line(
).encode(
    x=alt.X('t:Q'),
    y=alt.Y('dB:Q'),
    color=alt.Color('signal'),
)

In [72]:
(x_y | w_db | w_phase
).resolve_scale(y="independent"
).properties(
    title='Butterworth'
)

alt.HConcatChart(...)

In [79]:
w_lp

array([    0.        ,   119.40298507,   238.80597015,   358.20895522,
         477.6119403 ,   597.01492537,   716.41791045,   835.82089552,
         955.2238806 ,  1074.62686567,  1194.02985075,  1313.43283582,
        1432.8358209 ,  1552.23880597,  1671.64179104,  1791.04477612,
        1910.44776119,  2029.85074627,  2149.25373134,  2268.65671642,
        2388.05970149,  2507.46268657,  2626.86567164,  2746.26865672,
        2865.67164179,  2985.07462687,  3104.47761194,  3223.88059701,
        3343.28358209,  3462.68656716,  3582.08955224,  3701.49253731,
        3820.89552239,  3940.29850746,  4059.70149254,  4179.10447761,
        4298.50746269,  4417.91044776,  4537.31343284,  4656.71641791,
        4776.11940299,  4895.52238806,  5014.92537313,  5134.32835821,
        5253.73134328,  5373.13432836,  5492.53731343,  5611.94029851,
        5731.34328358,  5850.74626866,  5970.14925373,  6089.55223881,
        6208.95522388,  6328.35820896,  6447.76119403,  6567.1641791 ,
      